In [ ]:
# Code written by: Michael Bramble | michael.s.bramble@jpl.nasa.gov
# script for testing loading and overlay of USGS USMIN shapefiles
# 20240229 - initial version

In [34]:
import geopandas as gpd
import geodatasets
import contextily as cx
# import earthaccess
import os
# from osgeo import gdal
# import numpy as np
# import pandas as pd
# import xarray as xr
# from scipy.optimize import lsq_linear
import matplotlib.pyplot as plt
# import math
# import rasterio as rio
# import holoviews as hv
# import hvplot.xarray
# import netCDF4 as nc
# import sys
# sys.path.append('/Users/bramble/My Drive/_JPL_AMD/EMIT-Data-Resources-main/python/modules/')
# from emit_tools import emit_xarray
# import emit_tools

In [ ]:
shapefile_024k_points = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_24k_Points.shp")
shapefile_024k_polygons = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_24k_Polygons.shp")
shapefile_048k_points = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_48k_Points.shp")
shapefile_048k_polygons = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_48k_Polygons.shp")
shapefile_625k_points = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_625k_Points.shp")
shapefile_625k_polygons = gpd.read_file("/Users/bramble/Documents/emit/gis/USGS_TopoMineSymbols_ver10_Shapefiles/USGS_TopoMineSymbols_625k_Polygons.shp")
print(shapefile_024k_points)

In [ ]:
# project shapefiles

# https://spatialreference.org/ref/epsg/4326/
proj = 4326
shapefile_625k_points = shapefile_625k_points.to_crs(proj)

In [ ]:
# shapefile_625k_points.plot()

# max_lat = 37.94
# min_lat = 37.89
# max_lon = -107.70
# min_lon = -107.65
north = 37.94
south = 37.89
west = -107.70
east = -107.65

shapefile_625k_points_clip = shapefile_625k_points.clip_by_rect(west, south, east, north)
ax = shapefile_625k_points_clip.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
# cx.add_basemap(ax, crs=shapefile_625k_points.crs)
# cx.add_basemap(ax, crs=shapefile_625k_points.crs, source=cx.providers.CartoDB.Positron)
cx.add_basemap(ax, crs=shapefile_625k_points.crs, source=cx.providers.USGS.USTopo)

In [23]:
from owslib.wms import WebMapService
wms = WebMapService('https://mrdata.usgs.gov/services/wfs/usmin', version='1.3.0')

/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/owslib/map/wms130.py:122: UserWarning: Content metadata for layer "points" already exists. Using child layer
  warnings.warn('Content metadata for layer "%s" already exists. Using child layer' % cm.id)


In [59]:
# wms.identification.version
# wms.identification.title
# wms.identification.abstract
# list(wms.contents)
# wms['points'].title
# wms['points'].styles
# [op.name for op in wms.operations]
wms.getOperationByName('GetMap').formatOptions



['image/png',
 'image/jpeg',
 'image/png; mode=8bit',
 'image/vnd.jpeg-png',
 'image/vnd.jpeg-png8',
 'application/x-pdf',
 'image/svg+xml',
 'image/tiff',
 'application/json']

In [60]:
import io

north = 37.94
south = 37.89
west = -107.70
east = -107.65

img = wms.getmap(layers=['points'],
                 styles=['default'],
                 srs='EPSG:4326',
                 bbox=(west, south, east, north),
                 size=(600, 600),
                 format='image/png',
                 transparent=True
                 )


image = io.BytesIO(img.read())
data = plt.imread(image)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0, 0, 1, 1])
ax.imshow(data,extent=(west, east, south, north))
plt.show()

ServiceException: msWMSLoadGetMapParams(): Image handling error. Unsupported output format (image/svg xml).

In [ ]:
from owslib.wfs import WebFeatureService

# URL for WFS backend
# url = "http://geo.stat.fi/geoserver/vaestoruutu/wfs"
url = "https://mrdata.usgs.gov/services/wfs/usmin"


# Initialize
wfs = WebFeatureService(url=url)

# Service provider 
# print(wfs.identification.title)

# Get WFS version
# print(wfs.version)

# # Available methods
# print([operation.name for operation in wfs.operations])

# # Available data layers
# print(list(wfs.contents))

# # Print all metadata of all layers
# for layer, meta in wfs.items():
#     print(meta.__dict__)


In [ ]:
# Read the data into GeoDataFrame:

import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

# URL for WFS backend
# url = "http://geo.stat.fi/geoserver/vaestoruutu/wfs"
url = "https://mrdata.usgs.gov/services/wfs/usmin"


# Initialize
wfs = WebFeatureService(url=url)

# Fetch the last available layer (as an example) --> 'vaestoruutu:vaki2021_5km'
layer_name = list(wfs.contents)[-1]

# Specify the parameters for fetching the data
# Count: specificies amount of rows to return (e.g. 10000 or 100)
# startIndex: specifies at which offset to start returning rows
params = dict(service='WFS', version="1.1.0", request='GetFeature',
      typeName=layer_name, outputFormat='json', count=100, startIndex=0)

# Parse the URL with parameters
wfs_request_url = Request('GET', url, params=params).prepare().url

# Read data from URL
data = gpd.read_file(wfs_request_url)



In [ ]:
import geopandas as gp
import requests
from fiona import BytesCollection    

# def wfs2gp_df(layer_name, url, bbox=None, wfs_version="2.0.0", outputFormat='application/gml+xml; version=3.2'):
#     params = dict(service='WFS', version=wfs_version,request='GetFeature', typeName=layer_name, outputFormat=outputFormat)
#     with BytesCollection(requests.get(url,params=params).content) as f:
#         df = gp.GeoDataFrame.from_features(f)
#     return df

def wfs2gp_df(layer_name, url, bbox=None, wfs_version="1.1.0", outputFormat='application/gml+xml; version=3.2'):
    params = dict(service='WFS', version=wfs_version,request='GetFeature', typeName=layer_name, outputFormat=outputFormat)
    with BytesCollection(requests.get(url,params=params).content) as f:
        df = gp.GeoDataFrame.from_features(f)
    return df

# wfs2gp_df('your_wfs_layer_name', 'your_wfs_url').head()

# wfs2gp_df('usmin', 'https://mrdata.usgs.gov/services/wfs/usmin').head()

In [ ]:
import geopandas as gpd
import requests
import geojson
# from pyproj import CRS
from owslib.wfs import WebFeatureService

# Specify the url for the backend.
# Here we are using data from Statistics Finland: https://www.stat.fi/org/avoindata/paikkatietoaineistot_en.html. (CC BY 4.0)
url = "https://mrdata.usgs.gov/services/wfs/usmin"

# Specify parameters (read data in json format).
params = dict(
    service="WFS",
    version="1.1.0",
    request="GetFeature",
    typeName="tilastointialueet:kunta4500k",
    outputFormat="json",
)

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson and set coordinate reference system
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content), crs="EPSG:4326")

In [30]:
#FOR WMS data
# https://geographyplanning.buffalostate.edu/wMix/python/Geopandas%20Basics%20using%20Open%20Data.html

import os
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

shp_url="https://mrdata.usgs.gov/services/usmin"
shp_gdf=gpd.read_file(shp_url)
shp_gdf.plot(column='points', figsize=(9,9));

DriverError: '/vsimem/1b7eb6af723f41dc9944efd667744d00' not recognized as a supported file format.